In [5]:
from PIL import Image
import os
from collections import defaultdict
from pathlib import Path

In [4]:
def categorize_images_by_size(input_folder):
    """
    Analyze images and categorize them by size ranges.
    Creates a frequency table showing distribution across size categories.
    
    Args:
        input_folder: Path to folder containing images
    """
    
    # Define size categories (width x height)
    categories = [
        ("< 24x24", 0, 24),
        ("24x24 - 32x32", 24, 32),
        ("32x32 - 64x64", 32, 64),
        ("64x64 - 128x128", 64, 128),
        ("128x128 - 256x256", 128, 256),
        ("256x256 - 512x512", 256, 512),
        ("> 512x512", 512, float('inf'))
    ]
    
    # Track counts and store image details
    category_counts = defaultdict(int)
    category_images = defaultdict(list)
    
    # Supported image formats
    valid_extensions = {'.png', '.jpg', '.jpeg'}
    
    total_images = 0
    
    print("Analyzing images...\n")
    
    # Process each image
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        
        if not os.path.isfile(file_path):
            continue
            
        ext = os.path.splitext(filename)[1].lower()
        if ext not in valid_extensions:
            continue
        
        try:
            img = Image.open(file_path)
            width, height = img.size
            max_dimension = max(width, height)
            
            # Find appropriate category
            for cat_name, min_size, max_size in categories:
                if min_size <= max_dimension < max_size:
                    category_counts[cat_name] += 1
                    category_images[cat_name].append(f"{filename} ({width}x{height})")
                    break
            
            total_images += 1
            
        except Exception as e:
            print(f"✗ Error reading {filename}: {str(e)}")
    
    # Print frequency table
    print("=" * 70)
    print("IMAGE SIZE DISTRIBUTION")
    print("=" * 70)
    print(f"{'Category':<20} {'Count':>10} {'Percentage':>12} {'Bar':>20}")
    print("-" * 70)
    
    for cat_name, min_size, max_size in categories:
        count = category_counts[cat_name]
        percentage = (count / total_images * 100) if total_images > 0 else 0
        
        # Create visual bar
        bar_length = int(percentage / 5)  # Scale: 5% = 1 char
        bar = "█" * bar_length
        
        print(f"{cat_name:<20} {count:>10} {percentage:>11.1f}% {bar:>20}")
    
    print("-" * 70)
    print(f"{'TOTAL':<20} {total_images:>10} {100.0:>11.1f}%")
    print("=" * 70)
    
    # Print detailed breakdown (optional)
    print("\n" + "=" * 70)
    print("DETAILED BREAKDOWN BY CATEGORY")
    print("=" * 70)
    
    for cat_name, min_size, max_size in categories:
        if category_counts[cat_name] > 0:
            print(f"\n{cat_name}: {category_counts[cat_name]} images")
            print("-" * 70)
            for img_info in category_images[cat_name][:10]:  # Show first 10
                print(f"  • {img_info}")
            
            if len(category_images[cat_name]) > 10:
                remaining = len(category_images[cat_name]) - 10
                print(f"  ... and {remaining} more")
    
    print("\n" + "=" * 70)
    
    return category_counts, total_images


# Usage example
if __name__ == "__main__":
    input_folder = "./downloads"
    
    if not os.path.exists(input_folder):
        print(f"Error: Folder '{input_folder}' not found!")
        print("Please update the input_folder path.")
    else:
        categorize_images_by_size(input_folder)

Analyzing images...

IMAGE SIZE DISTRIBUTION
Category                  Count   Percentage                  Bar
----------------------------------------------------------------------
< 24x24                      47         2.2%                     
24x24 - 32x32                 9         0.4%                     
32x32 - 64x64               198         9.3%                    █
64x64 - 128x128             545        25.6%                █████
128x128 - 256x256           782        36.7%              ███████
256x256 - 512x512           387        18.2%                  ███
> 512x512                   160         7.5%                    █
----------------------------------------------------------------------
TOTAL                      2128       100.0%

DETAILED BREAKDOWN BY CATEGORY

< 24x24: 47 images
----------------------------------------------------------------------
  • 002-lospec-dailies-286456.png (16x16)
  • fresno-nightcrawler-082838.png (16x15)
  • green-slime-040105.png (16x1

In [7]:
def process_pixel_art(input_folder, output_folder, target_size=128):
    # Create output folder if it doesn't exist
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    # Supported image formats
    valid_extensions = {'.png', '.jpg', '.jpeg'}
    
    # Process each image in the input folder
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        
        # Skip if not a file or not a valid image
        if not os.path.isfile(file_path):
            continue
            
        ext = os.path.splitext(filename)[1].lower()
        if ext not in valid_extensions:
            continue
        
        try:
            # Open image
            img = Image.open(file_path)
            
            # Handle transparency: replace with black if PNG has alpha channel
            if ext == '.png' and (img.mode == 'RGBA' or img.mode == 'LA' or 'transparency' in img.info):
                # Create black background
                background = Image.new('RGB', img.size, (0, 0, 0))
                # Convert image to RGBA if needed
                if img.mode != 'RGBA':
                    img = img.convert('RGBA')
                # Composite image over black background
                background.paste(img, mask=img.split()[3])  # Use alpha channel as mask
                img = background
            else:
                # Convert to RGB for consistent color fidelity
                if img.mode != 'RGB':
                    img = img.convert('RGB')
            
            width, height = img.size
            
            # Skip images larger than 512x512
            if width > 512 or height > 512:
                print(f"⊗ Skipped: {filename} ({width}x{height} - exceeds 512x512 limit)")
                continue
            
            # Case 1: Image needs to be cropped (larger than target)
            if width > target_size or height > target_size:
                # Calculate crop box to center the crop
                left = (width - target_size) // 2
                top = (height - target_size) // 2
                right = left + target_size
                bottom = top + target_size
                
                # Adjust if image is not square
                if width < target_size:
                    left = 0
                    right = width
                if height < target_size:
                    top = 0
                    bottom = height
                
                img = img.crop((left, top, right, bottom))
                
                # If after crop we're still not target size, scale
                if img.size != (target_size, target_size):
                    img = img.resize((target_size, target_size), Image.NEAREST)
            
            # Case 2: Image needs to be scaled up (smaller than target)
            elif width < target_size or height < target_size:
                # Use NEAREST neighbor to maintain sharp pixel art edges
                img = img.resize((target_size, target_size), Image.NEAREST)
            
            # Case 3: Image is already the target size
            # (no action needed)
            
            # Save processed image
            output_path = os.path.join(output_folder, filename)
            
            # Save as PNG to preserve color fidelity (no compression artifacts)
            if ext != '.png':
                output_path = os.path.splitext(output_path)[0] + '.png'
            
            img.save(output_path, 'PNG', optimize=False)
            print(f"✓ Processed: {filename} ({width}x{height} → {target_size}x{target_size})")
            
        except Exception as e:
            print(f"✗ Error processing {filename}: {str(e)}")
            
input_folder = "./downloads"
output_folder = "./final"

process_pixel_art(input_folder, output_folder, target_size=128)
print("\nProcessing complete!")

✓ Processed: coming-soon-017725.png (128x160 → 128x128)
✓ Processed: first-day-of-school-466173.png (166x333 → 128x128)
✓ Processed: planet-995488.png (32x32 → 128x128)
✓ Processed: windy-day-253632.png (392x240 → 128x128)
⊗ Skipped: 013ans-433589.png (640x688 - exceeds 512x512 limit)
✓ Processed: beast-842500.png (64x64 → 128x128)
✓ Processed: 002-lospec-dailies-286456.png (16x16 → 128x128)
✓ Processed: phantom-blood-8bit-185693.png (256x224 → 128x128)
✓ Processed: dreamland-583903.png (297x445 → 128x128)
✓ Processed: substantiation-956675.png (128x128 → 128x128)
✓ Processed: simon-the-digger-989691.png (64x64 → 128x128)
✓ Processed: from-orbit-822444.png (200x200 → 128x128)
✓ Processed: night-window-628578.png (128x128 → 128x128)
⊗ Skipped: 014ans-181630.png (640x688 - exceeds 512x512 limit)
✓ Processed: niche-227510.png (128x64 → 128x128)
✓ Processed: kilrogg-deadeye-009845.png (132x132 → 128x128)
✓ Processed: pixel-art-freebie-for-matingcall-on-gaiaonline-837764.png (64x64 → 128x12